In [1]:
import urllib.request
import json
import datetime
# HTTP ヘッダの　RFC2822　日付用に
from email.utils import parsedate_to_datetime

In [1]:
# Test: Get public info
endPoint_bf='https://api.bitflyer.jp'

def getResponse_bf(path):
    global endPoint_bf
    url=endPoint_bf+path
    response=urllib.request.urlopen(url)
    res_date=parsedate_to_datetime(response.headers['date'])
    content = json.loads(response.read().decode('utf8'))
    return res_date,content

def getResponse_private_bf(path):
    pass


In [37]:
# Public なもの
# 取り扱いマーケットの取得
def getMarkets_bf():
    path_Markets='/v1/markets'
    getResponse_bf(path_Markets)
    res_date,json_data = getResponse_bf(path_Markets)
    return res_date,json_data

res_date,markets=getMarkets_bf()
print("UTC 日時:{0:%Y-%m-%d %H:%M:%S}".format(res_date))
markets

UTC 日時:2017-08-19 01:28:32


[{'product_code': 'BTC_JPY'},
 {'product_code': 'FX_BTC_JPY'},
 {'product_code': 'ETH_BTC'},
 {'product_code': 'BCH_BTC'},
 {'alias': 'BTCJPY_MAT1WK', 'product_code': 'BTCJPY25AUG2017'},
 {'alias': 'BTCJPY_MAT2WK', 'product_code': 'BTCJPY01SEP2017'}]

In [26]:
print(response.headers['date'])
t=parsedate_to_datetime(response.headers['date'])
print(t)


Sat, 19 Aug 2017 00:34:49 GMT
2017-08-19 00:34:49+00:00


In [38]:
# マーケットごとの板(取引状況)をとる
def getBoard_bf(product_code):
    global endPoint_bf    
    path_api='/v1/board'
    query='?product_code='+product_code
    path=path_api+query
    res_date,board = getResponse_bf(path)
    return res_date,board

res_date,board_BTC_JPY=getBoard_bf('BTC_JPY')
print("日時:{0:%Y-%m-%d %H:%M:%S}".format(res_date))
board_BTC_JPY

日時:2017-08-19 01:35:49


{'asks': [{'price': 460799.0, 'size': 0.95552163},
  {'price': 460800.0, 'size': 0.81},
  {'price': 461310.0, 'size': 2.0},
  {'price': 461314.0, 'size': 0.002},
  {'price': 461372.0, 'size': 0.79},
  {'price': 461460.0, 'size': 1.0},
  {'price': 461469.0, 'size': 2.0},
  {'price': 461510.0, 'size': 5.1},
  {'price': 461553.0, 'size': 0.11966012},
  {'price': 461617.0, 'size': 0.002},
  {'price': 461640.0, 'size': 4.1},
  {'price': 461696.0, 'size': 0.02173987},
  {'price': 461699.0, 'size': 0.01163969},
  {'price': 461750.0, 'size': 0.03},
  {'price': 461795.0, 'size': 2.0},
  {'price': 461850.0, 'size': 0.9},
  {'price': 461900.0, 'size': 0.04},
  {'price': 461980.0, 'size': 9.5},
  {'price': 462000.0, 'size': 1.25},
  {'price': 462084.0, 'size': 2.0},
  {'price': 462200.0, 'size': 24.72},
  {'price': 462222.0, 'size': 0.002},
  {'price': 462299.0, 'size': 0.01163969},
  {'price': 462300.0, 'size': 35.61},
  {'price': 462338.0, 'size': 0.353},
  {'price': 462350.0, 'size': 2.0},
  {'

In [5]:
# マーケットごとのTickerをとる
def getTicker_bf(product_code):
    global endPoint_bf
    path='/v1/ticker'
    query='?product_code='+product_code
    url=endPoint_bf+path+query
    response=urllib.request.urlopen(url)
    board = json.loads(response.read().decode('utf8'))
    return board


print("日時:{0:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now()))
ticker_BTC_JPY=getTicker_bf('BTC_JPY')
print("売り:{0}".format(ticker_BTC_JPY.get('best_ask')))
print("買い:{0}".format(ticker_BTC_JPY.get('best_bid')))

日時:2017-08-09 16:47:00
売り:382400.0
買い:382370.0


In [6]:
# Private なもの
api_key_bf='YOUR_API_KEY'

#import base64
#api_secret_bf=base64.b64decode(b'YOUR_API_SECRETTxY=')
api_secret_bf=b'YOUR_API_SECRETTxY='

# sha256 署名 hmac が必要なので暗号化ライブラリのインポート
import hmac
import hashlib

In [7]:
# 自分の所有資産確認
def getMyBalance_bf():
    global api_key_bf, api_secret_bf, endPoint_bf
    
    timestamp = "{0:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())
    method = "GET"
    path = "/v1/me/getbalance"

    text=(timestamp + method + path).encode('utf-8')
    
    m=hmac.new(api_secret_bf,text,hashlib.sha256)
    signature=m.hexdigest()
    
    headers = {
        'ACCESS-KEY':api_key_bf,
        'ACCESS-TIMESTAMP':timestamp,
        'ACCESS-SIGN':signature
    }
    
    url=endPoint_bf+path
    
    req = urllib.request.Request(url, None, headers)
    response = urllib.request.urlopen(req)
    
    balance = json.loads(response.read().decode('utf8'))
    return balance

myBalance=getMyBalance_bf()
myBalance

[{'amount': 869.0, 'available': 869.0, 'currency_code': 'JPY'},
 {'amount': 0.200005, 'available': 0.200005, 'currency_code': 'BTC'},
 {'amount': 0.119985, 'available': 0.119985, 'currency_code': 'BCH'},
 {'amount': 0.8, 'available': 0.8, 'currency_code': 'ETH'},
 {'amount': 0.0, 'available': 0.0, 'currency_code': 'ETC'},
 {'amount': 0.0, 'available': 0.0, 'currency_code': 'LTC'}]

In [ ]:
# 自分の注文記録確認
def getMyHistoryOrder_bf():
    global api_key_bf, api_secret_bf, endPoint_bf
        
    timestamp = "{0:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())
    method = "GET"
    path = "/v1/me/getchildorders"

    text=(timestamp + method + path).encode('utf-8')
    
    m=hmac.new(api_secret_bf,text,hashlib.sha256)
    signature=m.hexdigest()
    
    headers = {
        'ACCESS-KEY':api_key_bf,
        'ACCESS-TIMESTAMP':timestamp,
        'ACCESS-SIGN':signature
    }
    
    url=endPoint_bf+path
    
    req = urllib.request.Request(url, None, headers)
    response = urllib.request.urlopen(req)
    
    history = json.loads(response.read().decode('utf8'))
    return history
MyHistoryOrder=getMyHistoryOrder_bf()
MyHistoryOrder

In [21]:
# 自分のコイン入履歴記録確認
def getMyHistoryCoinIns_bf():
    global api_key_bf, api_secret_bf, endPoint_bf
        
    timestamp = "{0:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())
    method = "GET"
    path = "/v1/me/getcoinins"

    text=(timestamp + method + path).encode('utf-8')
    
    m=hmac.new(api_secret_bf,text,hashlib.sha256)
    signature=m.hexdigest()
    
    headers = {
        'ACCESS-KEY':api_key_bf,
        'ACCESS-TIMESTAMP':timestamp,
        'ACCESS-SIGN':signature
    }
    
    url=endPoint_bf+path
    
    req = urllib.request.Request(url, None, headers)
    response = urllib.request.urlopen(req)
    
    coinIns = json.loads(response.read().decode('utf8'))
    return coinIns
MyHistoryCoinIns=getMyHistoryCoinIns_bf()
MyHistoryCoinIns

[{'address': '0x201708BCH',
  'amount': 0.119985,
  'currency_code': 'BCH',
  'event_date': '2017-08-02T03:33:23.01',
  'id': 1028730,
  'order_id': 'CDP20170802-033323-207527',
  'status': 'COMPLETED',
  'tx_hash': '0x201708BCH'}]

In [13]:
dir(MyHistoryCoinIns )

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']